THEME: The project helps passengers and bike drivers find the most convenient pickup point by analyzing real map data, calculating walking and driving distances, and choosing a location that minimizes travel effort for both sides in real time.

✔ osmnx — download live map data (roads, junctions)

✔ networkx — calculate shortest paths for walking + driving

✔ folium — display map with markers

✔ geopy — convert place names to coordinates

✔ ortools — optimize and select best pickup point

In [ ]:
!pip install -q osmnx ortools folium shapely rtree gradio geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.


Imports cell → Loads modules needed for maps, distance calculations, optimization, UI, and geocoding services.

In [ ]:
import math
import warnings
from typing import List, Dict, Tuple, Optional
import gradio as gr
import osmnx as ox
import networkx as nx
from osmnx.features import features_from_point
import folium
from shapely.geometry import Point
from ortools.linear_solver import pywraplp
from geopy.geocoders import Nominatim
warnings.filterwarnings("ignore")
ox.settings.use_cache = True
ox.settings.log_console = False

haversine_dist_m() → Computes straight-line distance between two coordinates when road paths are unavailable.

nearest_node() → Finds the closest road intersection to any location for routing.

In [ ]:
def haversine_dist_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = math.radians(float(lat1))
    phi2 = math.radians(float(lat2))
    dphi = math.radians(float(lat2) - float(lat1))
    dlambda = math.radians(float(lon2) - float(lon1))
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def nearest_node(G, point: Tuple[float,float]):
    return ox.distance.nearest_nodes(G, point[1], point[0])

geocode_place() → Converts user-entered place names or addresses into latitude-longitude values.

In [ ]:
geolocator = Nominatim(user_agent="optiguide_app_v1")

def geocode_place(place_name: str) -> Optional[Tuple[float,float]]:
    if not place_name or place_name.strip()=="":
        return None
    try:
        loc = geolocator.geocode(place_name, timeout=10)
        if loc:
            return (loc.latitude, loc.longitude)
    except:
        return None
    return None

load_area_graph() → Downloads walking/driving roads and nearby landmarks from OpenStreetMap around the passenger area.

In [ ]:
def load_area_graph(lat, lon, dist_m=500):
    G_drive = ox.graph_from_point((lat, lon), dist=dist_m, network_type="drive")
    G_walk = ox.graph_from_point((lat, lon), dist=dist_m, network_type="walk")
    tags = {"amenity": True, "shop": True, "tourism": True}
    pois = features_from_point((lat, lon), tags=tags, dist=dist_m)
    return G_drive, G_walk, pois

generate_candidates() → Selects nearby intersections and landmarks as possible pickup options.

In [ ]:
def generate_candidates(G_drive, G_walk, pois, rider_lat, rider_lon, max_candidates=12):
    candidates = []
    nodes_gdf = ox.graph_to_gdfs(G_drive, nodes=True, edges=False)
    nodes_gdf["dist_to_rider"] = nodes_gdf.geometry.apply(
        lambda p: haversine_dist_m(rider_lat, rider_lon, p.y, p.x)
    )
    nodes_sorted = nodes_gdf.sort_values("dist_to_rider").head(max_candidates*2)

    for idx, row in nodes_sorted.iterrows():
        candidates.append({
            "type": "intersection",
            "name": f"Intersection {idx}",
            "lat": float(row.geometry.y),
            "lon": float(row.geometry.x),
            "node_id": idx
        })
        if len(candidates) >= max_candidates:
            break

    # Nearby POIs (landmarks)
    poi_count = 0
    for _, row in pois.iterrows():
        if poi_count >= 6 or len(candidates) >= max_candidates:
            break
        geom = row.geometry
        if isinstance(geom, Point):
            name = row.get("name") or row.get("amenity") or row.get("shop") or "POI"
            candidates.append({
                "type": "poi",
                "name": str(name),
                "lat": float(geom.y),
                "lon": float(geom.x),
                "node_id": None
            })
            poi_count += 1

    # Remove duplicates
    uniq = []
    seen = set()
    for c in candidates:
        key = (round(c["lat"],6), round(c["lon"],6))
        if key not in seen:
            uniq.append(c)
            seen.add(key)

    return uniq[:max_candidates]


compute_distances() → Calculates the rider’s walking distance and driver’s driving distance to each candidate point.

In [ ]:
def compute_distances(G_drive, G_walk, rider, driver, candidates):
    rider_lat, rider_lon = float(rider[0]), float(rider[1])
    driver_lat, driver_lon = float(driver[0]), float(driver[1])

    try:
        rider_walk_node = nearest_node(G_walk, (rider_lat, rider_lon))
    except:
        rider_walk_node = None

    try:
        driver_node = nearest_node(G_drive, (driver_lat, driver_lon))
    except:
        driver_node = None

    results = []

    for c in candidates:
        # WALKING
        try:
            walk_node = nearest_node(G_walk, (c["lat"], c["lon"]))
            walk_dist = nx.shortest_path_length(G_walk, rider_walk_node, walk_node, weight="length")
        except:
            walk_dist = haversine_dist_m(rider_lat, rider_lon, c["lat"], c["lon"])

        # DRIVING
        try:
            cand_node = c["node_id"] if c["node_id"] else nearest_node(G_drive, (c["lat"], c["lon"]))
            drive_dist = nx.shortest_path_length(G_drive, driver_node, cand_node, weight="length")
        except:
            drive_dist = haversine_dist_m(driver_lat, driver_lon, c["lat"], c["lon"])

        results.append({
            **c,
            "walk_m": float(walk_dist),
            "drive_m": float(drive_dist)
        })

    return results

select_best_pickup() → Uses OR-Tools to choose the pickup point with the lowest combined travel cost.

In [ ]:
def select_best_pickup(candidates, weight_drive=0.7, weight_walk=0.3):
    n = len(candidates)
    solver = pywraplp.Solver.CreateSolver("SCIP")
    x = [solver.IntVar(0,1,f"x{i}") for i in range(n)]
    solver.Add(sum(x) == 1)

    objective = solver.Objective()
    for i, c in enumerate(candidates):
        score = weight_drive*c["drive_m"] + weight_walk*c["walk_m"]
        objective.SetCoefficient(x[i], score)
    objective.SetMinimization()

    status = solver.Solve()
    if status in (pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE):
        for i in range(n):
            if x[i].solution_value() > 0.5:
                return i
    return None


Uses Google OR-Tools (linear optimization solver).

You give importance values:



-------------------------Driver weight (e.g., 0.7)--------------------------

This tells the AI how important the driver’s distance is in optimization.

Value range: 0.0 to 1.0


Higher driver weight (ex: 0.8 or 0.9)
→ AI will choose pickup point that is easy for bike owner


Lower driver weight (ex: 0.2 or 0.3)
→ AI will care less about driver travel


Real-world understanding:

If driver weight = 0.7

It means:

“Make the pickup easy for the driver (70% importance),
but also consider the passenger (30% importance).”




------------------------Passenger weight (e.g., 0.3)---------------------------

This tells the AI how important the passenger’s walking distance is.

Value range: 0.0 to 1.0

Higher passenger weight (0.6 or 0.7)
→ AI will try to reduce passenger walking

Lower passenger weight
→ Passenger walking is less important in decision

Real-world understanding:

Passenger weight = 0.3

It means:

AI will care about the passenger 30%
but prioritize the driver more (if driver weight = 0.7)

explain_pickup() → Creates a readable explanation showing why the selected pickup point is the best.

In [ ]:
def explain_pickup(c):
    return f"""
### ⭐ Best Pickup Point: **{c['name']}**

- 🚶 Rider walking: **{c['walk_m']:.0f} meters**
- 🏍️ Driver distance: **{c['drive_m']:.0f} meters**

📍 Coordinates: **{c['lat']:.6f}, {c['lon']:.6f}**

Done using real-time map + optimization.
"""

create_map_html() → Displays the rider, driver, and pickup points on an interactive map with colored markers.

In [ ]:
def create_map_html(rider, driver, candidates, chosen_idx):
    m = folium.Map(location=[rider[0], rider[1]], zoom_start=17)

    folium.Marker([rider[0], rider[1]], popup="Passenger", icon=folium.Icon(color="blue")).add_to(m)
    folium.Marker([driver[0], driver[1]], popup="Bike Owner", icon=folium.Icon(color="purple")).add_to(m)

    for i, c in enumerate(candidates):
        color = "red" if i == chosen_idx else "orange"
        folium.Marker([c["lat"], c["lon"]],
                      popup=f"{c['name']}<br>{int(c['walk_m'])}m walk | {int(c['drive_m'])}m drive",
                      icon=folium.Icon(color=color)).add_to(m)

    return m._repr_html_()

optimize_ui() → Connects all logic to the Gradio interface and returns the final pickup point, explanation, and map.

In [ ]:
def optimize_ui(place_name, rider_lat, rider_lon, driver_lat, driver_lon,
                auto_driver, radius_m, max_candidates, weight_drive, weight_walk):

    # Passenger location
    if place_name.strip():
        coords = geocode_place(place_name)
        if coords is None:
            return "Place not found. Try typing full address.", "", "", ""
        rider = coords
    else:
        try:
            rider = (float(rider_lat), float(rider_lon))
        except:
            return "Invalid passenger coordinates", "", "", ""

    # Driver location
    if driver_lat.strip() and driver_lon.strip():
        try:
            driver = (float(driver_lat), float(driver_lon))
        except:
            return "Invalid driver coordinates", "", "", ""
    else:
        if auto_driver:
            driver = (rider[0] + 0.001, rider[1] + 0.001)
        else:
            return "Please enter driver location or enable simulation", "", "", ""

    # Load map data
    try:
        G_drive, G_walk, pois = load_area_graph(rider[0], rider[1], dist_m=int(radius_m))
    except Exception as e:
        return f"Map error: {e}", "", "", ""

    # Generate candidates
    candidates = generate_candidates(G_drive, G_walk, pois, rider[0], rider[1], max_candidates)
    if not candidates:
        return "No candidates found", "", "", ""

    # Compute distances
    scored = compute_distances(G_drive, G_walk, rider, driver, candidates)

    # Select best
    best_idx = select_best_pickup(scored, weight_drive, weight_walk)
    if best_idx is None:
        return "No pickup point possible", "", "", ""

    chosen = scored[best_idx]
    explanation = explain_pickup(chosen)
    map_html = create_map_html(rider, driver, scored, best_idx)
    gmaps_link = f"https://www.google.com/maps?q={chosen['lat']},{chosen['lon']}"

    return explanation, f"{chosen['name']} ({chosen['lat']},{chosen['lon']})", map_html, gmaps_link


Gradio

Builds an interactive interface for searching, refreshing, and summarizing news.

Allows users to ask questions and receive answers powered by the stored articles.

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🚕 OptiGuide — Real-Time Rapido Pickup Optimizer")

    with gr.Row():
        with gr.Column():
            place_name = gr.Textbox(label="Place name (optional)", placeholder="e.g. 'MG Road Metro Bangalore'")
            rider_lat = gr.Textbox(label="Passenger Latitude")
            rider_lon = gr.Textbox(label="Passenger Longitude")

            driver_lat = gr.Textbox(label="Driver Latitude")
            driver_lon = gr.Textbox(label="Driver Longitude")
            auto_sim = gr.Checkbox(label="Simulate driver nearby", value=True)

            radius_m = gr.Number(label="Search radius (meters)", value=500)
            max_candidates = gr.Slider(4, 20, label="Max candidate points", value=10)
            weight_drive = gr.Slider(0.0, 1.0, label="Driver weight", value=0.7)
            weight_walk = gr.Slider(0.0, 1.0, label="Passenger weight", value=0.3)

            btn = gr.Button("Optimize Pickup 🚩")

        with gr.Column():
            explanation = gr.Markdown()
            chosen_text = gr.Textbox(label="Chosen Pickup Coordinates")
            map_html = gr.HTML()
            gm_link = gr.Textbox(label="Open in Google Maps")

    btn.click(
        optimize_ui,
        inputs=[place_name, rider_lat, rider_lon, driver_lat, driver_lon, auto_sim,
                radius_m, max_candidates, weight_drive, weight_walk],
        outputs=[explanation, chosen_text, map_html, gm_link]
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c475866e999619fdd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Enter Rider and driver:
  latitude and longitude

Step:1 Open Google Maps in your browser.

Step:2 Search for the place or zoom and move the map until you see the location you want.

Step:3 Right-click on that exact spot on the map.

Step:4 A small menu will appear.

Step:5 At the top of this menu, you’ll see a pair of numbers like 12.9716, 77.5946
these numbers are the latitude (first) and longitude (second) of that location.

NOTE: In Values dont put commas